<a href="https://colab.research.google.com/github/Rudra-prasad-tarai/nlpInternship/blob/main/POSTaggerMbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets seqeval scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=3739dc02abbe163eb83be853cba29daf6640b092559ccf6e521e30d1f9d4c70d
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
from datasets import load_dataset
dataset = load_dataset("LingoIITGN/COMI-LINGUA", "POS")  # Load POS task subset
train_data = dataset["train"]
test_data = dataset["test"]

In [3]:
dataset['train'][15683] # dataset['train'][i] the annotaion by all the annotators for the the sentence

{'Sentences': 'जैसलमेर - प्रधानमंत्री नरेंद्र मोदी (Narendra Modi) ने राजस्थान के जैसलमेर बॉर्डर पर लोंगेवाला पोस्ट पहुंच कर सेना के जवानों के साथ दिवाली (Diwali) का त्योहार मनाया।',
 'Predicted Tags': "[{'word': 'जैसलमेर', 'entity': 'PROPN'}, {'word': '-', 'entity': 'X'}, {'word': 'प्रधानमंत्री', 'entity': 'PROPN'}, {'word': 'नरेंद्र', 'entity': 'PROPN'}, {'word': 'मोदी', 'entity': 'PROPN'}, {'word': '(', 'entity': 'X'}, {'word': 'Narendra', 'entity': 'PROPN'}, {'word': 'Modi', 'entity': 'PROPN'}, {'word': ')', 'entity': 'X'}, {'word': 'ने', 'entity': 'VERB'}, {'word': 'राजस्थान', 'entity': 'PROPN'}, {'word': 'के', 'entity': 'ADP'}, {'word': 'जैसलमेर', 'entity': 'PROPN'}, {'word': 'बॉर्डर', 'entity': 'NOUN'}, {'word': 'पर', 'entity': 'ADP'}, {'word': 'लोंगेवाला', 'entity': 'ADJ'}, {'word': 'पोस्ट', 'entity': 'NOUN'}, {'word': 'पहुंच', 'entity': 'VERB'}, {'word': 'कर', 'entity': 'VERB'}, {'word': 'सेना', 'entity': 'NOUN'}, {'word': 'के', 'entity': 'ADP'}, {'word': 'जवानों', 'entity': '

In [4]:
import ast  # safely parse string to Python object

def extract_tokens_and_tags(example):
    ann = ast.literal_eval(example['Annotated by: Annotator 1 '])  # safely convert string to list of dicts
    tokens = [entry["word"] for entry in ann]
    tags = [entry["entity"] for entry in ann]
    return {"tokens": tokens, "tags": tags}


In [ ]:
# Convert to tokens and tags
processed_data = dataset.map(extract_tokens_and_tags)

# Keep only necessary columns
processed_data = processed_data.remove_columns([
    "Sentences", "Predicted Tags",
    "Annotated by: Annotator 1 ",
    "Annotated by: Annotator 2",
    "Annotated by: Annotator 3"
])


In [6]:
train_dataset = processed_data['train']
test_dataset = processed_data['test']

In [7]:
processed_data['train'][0]

{'tokens': ['Loan',
  'Apps',
  'की',
  'अब',
  'खैर',
  'नहीं',
  ',',
  'RBI',
  'ने',
  'बता',
  'दिया',
  'किस',
  '-',
  'किस',
  'पर',
  'होगी',
  'कार्रवाई',
  ',',
  'लिस्ट',
  'तैयार'],
 'tags': ['NOUN',
  'NOUN',
  'ADP',
  'ADV',
  'NOUN',
  'PART_NEG',
  'X',
  'PROPN',
  'ADP',
  'VERB',
  'VERB',
  'PRON_WH',
  'X',
  'PRON_WH',
  'ADP',
  'VERB',
  'NOUN',
  'X',
  'NOUN',
  'VERB']}

In [8]:
# Get all unique tags
unique_tags = list(set(tag for row in train_dataset for tag in row['tags']))
tag2id = {tag: i for i, tag in enumerate(sorted(unique_tags))}
id2tag = {i: tag for tag, i in tag2id.items()}
NUM_LABELS = len(tag2id)


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=NUM_LABELS,
    id2label=id2tag,
    label2id=tag2id
)


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128

    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_val = test_dataset.map(tokenize_and_align_labels, batched=True)


In [11]:
import transformers
print(transformers.__version__)


4.48.3


In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./muril-pos-results",
    # evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"
)


In [21]:
print(transformers.__file__)

/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [22]:
import numpy as np
from seqeval.metrics import accuracy_score, f1_score, classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2tag[label] for label in sent if label != -100]
        for sent in labels
    ]
    true_predictions = [
        [id2tag[pred] for (pred, label) in zip(sent_preds, sent_labels) if label != -100]
        for sent_preds, sent_labels in zip(predictions, labels)
    ]

    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions)
    }


In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    # evaluation_strategy="epoch",
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-23-616f034d73fe>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
t = trainer.train()


Step,Training Loss
50,0.285400
100,0.370400
150,0.428600
200,0.356300
250,0.372800
300,0.356000
350,0.342900
400,0.334400
450,0.335000
500,0.313700


In [ ]:
metrics = trainer.evaluate(eval_dataset=tokenized_val)
print(metrics)


In [27]:
metrics

{'eval_loss': 0.2083888202905655,
 'eval_accuracy': 0.936124181946022,
 'eval_f1': 0.897738487085089,
 'eval_runtime': 36.0356,
 'eval_samples_per_second': 138.724,
 'eval_steps_per_second': 17.344,
 'epoch': 3.0}